In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.base import clone
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

### Leyendo dataset base

In [3]:
x_train=pd.read_csv("X_train.csv")
x_test=pd.read_csv("X_test.csv")
y_train=pd.read_csv("Y_train.csv")
y_test=pd.read_csv("Y_test.csv")

In [4]:
x_train.head()

,Gender,Age,Height,Weight,BMI,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1,26.000000,1.720000,65.000000,21.971336,1,1,2.000000,3.000000,1,0,2.000000,0,0.000000,1.000000,1,4
1,0,26.000000,1.590000,47.000000,18.591037,1,1,2.000000,1.000000,1,0,2.000000,0,0.000000,2.000000,1,3
2,1,16.198153,1.691007,52.629374,18.405067,0,1,2.000000,2.000986,1,0,2.673835,0,0.992950,0.474836,1,3
3,1,24.481032,1.661277,90.744965,32.880501,1,1,1.712848,3.000000,1,0,1.162153,0,0.458981,0.885532,0,0
4,0,33.000000,1.560000,48.000000,19.723866,1,0,2.000000,3.000000,1,0,2.000000,0,1.000000,0.000000,1,3


In [5]:
x_train.drop(columns=["BMI"],inplace=True)

In [6]:
x_test.drop(columns=["BMI"],inplace=True)

In [7]:
len(x_train),len(x_test)

(1688, 423)

### Entrenamos con un RF rapidamente para analizar si se optiene el mismo baseline

In [11]:
rf = RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=2600, oob_score=True, class_weight = 'balanced', random_state=1, max_features='sqrt')
rf.fit(x_train, y_train)

C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=20, max_features='sqrt', n_estimators=2600,
                       oob_score=True, random_state=1)

In [12]:
rf.score(x_train, y_train)

1.0

In [13]:
rf.score(x_test, y_test)

0.9432624113475178

### Como en el paper no se menciona el tamaño del test no podemos tener un punto de comparacion igual pero se aplicara GridSearchCV  para obtener el mejor resultado usando CV

No usamos oob score para usar toda la data en el entrenamiento

In [16]:
#Entrenamos el modelo random forest de clasificación usando el GridSearchCV para considerar los mejores hiperparámetros
rf= RandomForestClassifier(random_state=42)

# grid searh to choose the best (combination of) hyperparameters, #'criterion': ['gini', 'entropy'], #'oob_score': [True, False], 'max_features':['sqrt', 0.1]

pg_rf={'criterion': ['gini', 'entropy'],'max_depth': [18, 20, 22], 'n_estimators': [2000,2600,2800], 
       'oob_score': [False], 'min_samples_split':[5,7] }

gs_rf= GridSearchCV (estimator= rf,
               param_grid= pg_rf,
               scoring='f1_macro',
               cv=5,refit=True)

# nested cross validation combining grid search (inner loop) and k-fold cv (outter loop)
gs_rf_scores = cross_val_score(gs_rf, X=x_train, y=y_train, cv=5, scoring='f1_macro', verbose = 10, n_jobs=-1)

# fit, and fit with best estimator
gs_rf.fit(x_train, y_train)

print('Train Accuracy:   {0:.1f}%'.format(gs_rf.score(x_train, y_train)*100))
print('CV Mean Accuracy: {0:.1f}%'.format(np.mean(gs_rf_scores)*100))
print('Test Accuracy:    {0:.1f}%'.format(gs_rf.score(x_test, y_test)*100))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 81.6min remaining: 122.4min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed: 81.9min remaining: 54.6min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 122.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 122.5min finished
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_trai

C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages

C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages

C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages

C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages

C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages

C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\omore\Anaconda3\envs\Machine_learning\lib\site-packages

Train Accuracy:   99.9%
CV Mean Accuracy: 94.5%
Test Accuracy:    94.7%


Train Accuracy:   99.9%
CV Mean Accuracy: 94.5%
Test Accuracy:    94.7%

In [17]:
gs_rf.best_estimator_

RandomForestClassifier(max_depth=18, min_samples_split=5, n_estimators=2600,
                       random_state=42)

In [28]:
rf= DecisionTreeClassifier(random_state=42,class_weight= "balanced")

# grid searh to choose the best (combination of) hyperparameters, #'criterion': ['gini', 'entropy'], #'oob_score': [True, False], 'max_features':['sqrt', 0.1]

pg_rf={'criterion': ['gini', 'entropy'],'max_depth': [6,8,10,12,16], 'min_samples_split':[2,3,4,6,7] }

gs_rf= GridSearchCV (estimator= rf,
               param_grid= pg_rf,
               scoring='f1_macro',
               cv=5,refit=True)

In [29]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [6, 8, 10, 12, 16],
                         'min_samples_split': [2, 3, 4, 6, 7]},
             scoring='f1_macro')

In [30]:
gs_rf_scores = cross_val_score(gs_rf, X=x_train, y=y_train, cv=5, scoring='f1_macro', verbose = 10, n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   12.5s remaining:   18.8s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   12.6s remaining:    8.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   19.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   19.2s finished


In [31]:
print('Train Accuracy:   {0:.1f}%'.format(gs_rf.score(x_train, y_train)*100))
print('CV Mean Accuracy: {0:.1f}%'.format(np.mean(gs_rf_scores)*100))
print('Test Accuracy:    {0:.1f}%'.format(gs_rf.score(x_test, y_test)*100))

Train Accuracy:   99.9%
CV Mean Accuracy: 94.1%
Test Accuracy:    94.6%


In [32]:
gs_rf.best_params_

{'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 3}

In [35]:
print(classification_report(list(y_test["NObeyesdad"]),gs_rf.predict(x_test)))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94        54
           1       0.85      0.95      0.89        58
           2       0.93      0.88      0.90        58
           3       0.95      0.97      0.96        58
           4       0.97      0.99      0.98        70
           5       0.97      0.97      0.97        60
           6       0.98      0.98      0.98        65

    accuracy                           0.95       423
   macro avg       0.95      0.95      0.95       423
weighted avg       0.95      0.95      0.95       423

